In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from config import gkey, yelp_key, census_key
from pprint import pprint
import requests

# Census API Key
c = Census(census_key, year=2013)

In [5]:
farm_markets = pd.read_csv("../farmers_markets.csv")

farm_markets.columns

Index(['FMID', 'MarketName', 'Website', 'Facebook', 'Twitter', 'Youtube',
       'OtherMedia', 'street', 'city', 'County', 'State', 'zip', 'Season1Date',
       'Season1Time', 'Season2Date', 'Season2Time', 'Season3Date',
       'Season3Time', 'Season4Date', 'Season4Time', 'x', 'y', 'Location',
       'Credit', 'WIC', 'WICcash', 'SFMNP', 'SNAP', 'Organic', 'Bakedgoods',
       'Cheese', 'Crafts', 'Flowers', 'Eggs', 'Seafood', 'Herbs', 'Vegetables',
       'Honey', 'Jams', 'Maple', 'Meat', 'Nursery', 'Nuts', 'Plants',
       'Poultry', 'Prepared', 'Soap', 'Trees', 'Wine', 'Coffee', 'Beans',
       'Fruits', 'Grains', 'Juices', 'Mushrooms', 'PetFood', 'Tofu',
       'WildHarvested', 'updateTime'],
      dtype='object')

In [6]:
farm_markets.head()

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,...,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,NaN,NaN,Danville,Caledonia,...,Y,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,NaN,NaN,NaN,6975 Ridge Road,Parma,Cuyahoga,...,N,N,Y,N,N,N,N,N,N,6/21/2017 5:15:01 PM
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,NaN,NaN,NaN,106 S. Main Street,Six Mile,NaN,...,N,N,N,N,N,N,N,N,N,2013
3,1010691,10th Steet Community Farmers Market,NaN,NaN,NaN,NaN,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,...,N,N,Y,N,N,N,N,N,N,10/28/2014 9:49:46 AM
4,1002454,112st Madison Avenue,NaN,NaN,NaN,NaN,NaN,112th Madison Avenue,New York,New York,...,N,N,N,N,N,N,N,N,N,3/1/2012 10:38:22 AM


In [7]:
# Drop the following columns to make a "clean" dataset

farm_markets_dropped = farm_markets.drop(["FMID", "Website", "Facebook", "Twitter", "Youtube",
                                       "OtherMedia", "street", "Location"], axis=1)
farm_markets_dropped.shape

(8730, 51)

In [8]:
# Rename x and y columns

farm_markets_newcols = farm_markets_dropped.rename(columns={"x":"Longitude",
                                                       "y":"Latitude"})
farm_markets_newcols.head()

,MarketName,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,...,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,Caledonia Farmers Market Association - Danville,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,NaN,...,Y,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM
1,Stearns Homestead Farmers' Market,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,NaN,NaN,NaN,...,N,N,Y,N,N,N,N,N,N,6/21/2017 5:15:01 PM
2,106 S. Main Street Farmers Market,Six Mile,NaN,South Carolina,29682,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,2013
3,10th Steet Community Farmers Market,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,NaN,NaN,NaN,...,N,N,Y,N,N,N,N,N,N,10/28/2014 9:49:46 AM
4,112st Madison Avenue,New York,New York,New York,10029,July to November,Tue:8:00 am - 5:00 pm;Sat:8:00 am - 8:00 pm;,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,3/1/2012 10:38:22 AM


In [9]:
farm_markets_newcols.count()

MarketName       8730
city             8690
County           8217
State            8730
zip              7793
Season1Date      5570
Season1Time      5733
Season2Date       455
Season2Time       444
Season3Date        82
Season3Time        79
Season4Date         6
Season4Time         6
Longitude        8701
Latitude         8701
Credit           8730
WIC              8730
WICcash          8730
SFMNP            8730
SNAP             8730
Organic          8730
Bakedgoods       8730
Cheese           8730
Crafts           8730
Flowers          8730
Eggs             8730
Seafood          8730
Herbs            8730
Vegetables       8730
Honey            8730
Jams             8730
Maple            8730
Meat             8730
Nursery          8730
Nuts             8730
Plants           8730
Poultry          8730
Prepared         8730
Soap             8730
Trees            8730
Wine             8730
Coffee           8730
Beans            8730
Fruits           8730
Grains           8730
Juices    

In [11]:
# Drop null lat/long: 29 farmers markets

farm_markets_validcoords = farm_markets_newcols.dropna(subset=['Latitude', 'Longitude'])

farm_markets_validcoords["Latitude"].count()

8701

In [12]:
# Set up URL to make Google API calls - replace null zipcodes
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
params = {"key":gkey}

# Make copy so as not to overwrite data
farm_markets_clean = farm_markets_validcoords.copy()

# Replace NaN zipcodes or zipcodes with too few digits
for index, row in farm_markets_validcoords.iterrows():
    if pd.isnull(row["zip"]) or (len(row["zip"]) != 5) or (row["zip"].isdigit() == False):
        params["latlng"] = f'{row["Latitude"]},{row["Longitude"]}'
        response = requests.get(base_url, params=params).json()
        
        # Find the response entry with zip codes, then add zip code to dataframe
        for info_dict in response["results"][0]["address_components"]:
            if "postal_code" in info_dict["types"]:
                farm_markets_clean.loc[index, "zip"] = info_dict["long_name"]

In [13]:
# Drop duplicate farmers market entries

farm_markets_df2 = farm_markets_clean.drop_duplicates(subset=['MarketName', 'zip'], keep='first')

In [14]:
farm_markets_df2.count()

MarketName       8631
city             8618
County           8145
State            8631
zip              8631
Season1Date      5505
Season1Time      5665
Season2Date       449
Season2Time       438
Season3Date        80
Season3Time        77
Season4Date         6
Season4Time         6
Longitude        8631
Latitude         8631
Credit           8631
WIC              8631
WICcash          8631
SFMNP            8631
SNAP             8631
Organic          8631
Bakedgoods       8631
Cheese           8631
Crafts           8631
Flowers          8631
Eggs             8631
Seafood          8631
Herbs            8631
Vegetables       8631
Honey            8631
Jams             8631
Maple            8631
Meat             8631
Nursery          8631
Nuts             8631
Plants           8631
Poultry          8631
Prepared         8631
Soap             8631
Trees            8631
Wine             8631
Coffee           8631
Beans            8631
Fruits           8631
Grains           8631
Juices    

In [15]:
regions = pd.read_csv("../Regions.csv")

df_market = farm_markets_clean.copy()
merge_df = df_market.merge(regions, left_on='State', right_on='State', how='outer')
merge_df.head()

,MarketName,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,...,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime,Region
0,Caledonia Farmers Market Association - Danville,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,NaN,...,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM,Northeast
1,Barre Farmers Market,Barre,Washington,Vermont,05641,05/31/2017 to 10/04/2017,Wed: 4:00 PM-7:30 PM;,NaN,NaN,NaN,...,N,Y,N,N,N,N,N,N,7/4/2017 10:33:01 AM,Northeast
2,Bethel Farmers Market,Bethel,Windsor,Vermont,05089,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,2009,Northeast
3,Bradford Farmers Market,Bradford,Orange,Vermont,05033,05/18/2014 to 10/19/2014,Sun: 10:00 AM-2:00 PM;,NaN,NaN,NaN,...,N,Y,N,N,N,N,N,N,4/9/2014 10:03:26 AM,Northeast
4,Brandon Farmers Market,Brandon,Rutland,Vermont,05733,05/22/2015 to 10/09/2015,Fri: 9:00 AM-2:00 PM;,NaN,NaN,NaN,...,N,Y,N,N,N,N,N,N,4/23/2015 3:34:19 PM,Northeast
